# Week 10 (Course 3, Week1). Assignment 1.

Представим, что международное круизное агентство "Carnival Cruise Line" решило себя разрекламировать с помощью баннеров и обратилось для этого к Вам.
Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего **20** штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с Вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры - легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях - Foursqare.

Часть открытых данных есть, например, на сайте archive.org: https://archive.org/details/201309_foursquare_dataset_umn
Скачаем любым удобным образом архив с этой страницы.


Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

|id     | user_id | venue_id | latitude   | longitude   | created_at          |
|-------|---------|----------|------------|-------------|---------------------|
|984301 | 2041916 | 5222     |            |             | 2012-04-21 17:39:01 |
|984222 | 15824   | 5222     | 38.8951118 | -77.0363658 | 2012-04-21 17:43:47 |
|984315 | 1764391 | 5222     |            |             | 2012-04-21 17:37:18 |
|984234 | 44652   | 5222     | 33.800745  | -84.41052   | 2012-04-21 17:43:43 |

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки не содержащие координат - они неинформативны для нас:

```
id,user_id,venue_id,latitude,longitude,created_at
984222,15824,5222,38.8951118,-77.0363658,2012-04-21T17:43:47
984234,44652,5222,33.800745,-84.41052,2012-04-21T17:43:43
984291,105054,5222,45.5234515,-122.6762071,2012-04-21T17:39:22
```

С помощью pandas построим DataFrame и убедимся, что все **396632** строк с координатами считаны успешно.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
from numpy.linalg import norm
from collections import Counter

In [2]:
data = pd.read_csv('chekins.csv', sep=',', header=0)

In [3]:
len(data)

396634

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача - хороший повод познакомиться с алгоритмом **MeanShift**, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в __sklearn user guide__ (http://scikit-learn.org/stable/modules/clustering.html#mean-shift), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте **MeanShift**, указав **bandwidth=0.1**, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [4]:
coords = data[['latitude', 'longitude']]
coords = coords[:100000]
coords.head()

,latitude,longitude
0,38.895112,-77.036366
1,33.800745,-84.410520
2,45.523452,-122.676207
3,40.764462,-111.904565
4,33.448377,-112.074037


In [5]:
ms = MeanShift(bandwidth=0.1)

In [6]:
ms.fit(coords)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

Примечание: на 396632 строках, кластеризация будет работать долго. Для получения корректного ответа достаточно и 100000 (~2 минуты на "среднем" ноутбуке). Быть очень терпеливым не возбраняется - результат от этого только улучшится.


Некоторые из получившихся кластеров содержат слишком мало точек - такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, **больше 15 элементов**. Центры этих кластеров и являются оптимальными для размещения.

In [7]:
min_thr = 15

In [9]:
labels = ms.labels_
cluster_centers = ms.cluster_centers_

In [10]:
label_to_count = Counter(labels)

In [11]:
cluster_nums_bigger_thr = [i for i, v in Counter(labels).iteritems() if v > min_thr]

In [12]:
big_clusters = cluster_centers[cluster_nums_bigger_thr] 

При желании увидеть получившиеся результаты на карте, можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт https://mapcustomizer.com имеет функцию **Bulk Entry**, куда можно вставить центры полученных кластеров в формате:

In [13]:
with open('out_big_entry.txt', 'wt') as f:
    f.writelines(['%f,%f\n' % (c[1], c[0]) for c in big_clusters])

In [14]:
centers = big_clusters

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу "Carnival Cruise Line" адреса офисов:

In [15]:
offices = [
    np.array([33.751277, -118.188740]), # 'Los Angeles'
    np.array([25.867736, -80.324116]),  # 'Miami'
    np.array([51.503016, -0.075479]),   # 'London'
    np.array([52.378894, 4.885084]),    # 'Amsterdam'
    np.array([39.366487, 117.036146]),  # 'Beijing'
    np.array([-33.868457, 151.205134])  # 'Sydney'
]

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [16]:
centers_dist = [
    min([(norm(office - center, ord=2), center) for office in offices], key=lambda x: x[0])
    for center in centers
]

sorted_centers_dist = sorted(centers_dist, key=lambda x:x[0])

_Примечание_: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико, чтобы не повлиять на кластеризацию или сортировку.

In [17]:
billboards = [c for d, c in sorted_centers_dist[:20]]

In [18]:
with open('billboards.txt', 'wt') as f:
    f.writelines(['%f,%f\n' % (c[0], c[1]) for c in billboards])

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании - широта и долгота этого центра, записанные через пробел.

## Несмотря на то, что в задаче требуется вводить 20 координат и только ближний, фактически выводить нужно только один и второй по удалённости

In [19]:
with open('assignment_1.txt', 'wt') as f:
    f.write(' '.join('%f %f' % (b[0], b[1]) for b in billboards[1:2]))

In [20]:
%cat assignment_1.txt

52.372964 4.892317